### Advertising

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [ ]:
df =  pd.read_csv('datasets/raw/Advertising.csv', sep=',', encoding='utf-8')
df.head(10)

In [ ]:
lm = smf.ols(formula="Sales~TV+Newspaper+Radio", data = df).fit()
lm.rsquared

#### Bondad de Ajsute más alta 

In [ ]:
lm = smf.ols(formula="Sales~TV", data = df).fit()
lm.rsquared

In [ ]:
lm = smf.ols(formula="Sales~Newspaper", data = df).fit()
lm.rsquared

In [ ]:
lm = smf.ols(formula="Sales~Radio", data = df).fit()
lm.rsquared

#### Insurance.csv

In [ ]:
df2 = pd.read_csv('datasets/raw/insurance.csv', sep=',', encoding='utf-8')
df2.head(10)

In [ ]:
# Mapear valores de la columna 'sex'
df2['sex'] = df2['sex'].map({'female': 1, 'male': 2})

# Mapear valores de la columna 'smoker'
df2['smoker'] = df2['smoker'].map({'yes': 1, 'no': 0})

In [ ]:
lm2 = smf.ols(formula="charges~age+sex+bmi+children+smoker", data = df2).fit()
lm2.params

### ML_houses_dataset

In [ ]:
df3= pd.read_csv('datasets/raw/ML_Houses_dataset.csv', sep=',', encoding='utf-8')
df3.head()

In [ ]:
df3.info()

In [ ]:
# Obtener las columnas numéricas y categóricas
numeric_columns = df3.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df3.select_dtypes(include=['object']).columns
# Convertir las columnas categóricas a tipo str
df3[categorical_columns] = df3[categorical_columns].astype(str)

# Definir las transformaciones para las columnas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Definir las transformaciones para las columnas categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Crear la columna transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Crear la pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Aplicar la pipeline al DataFrame
transformed_df = pipeline.fit_transform(df3)

In [ ]:
transformed_df

In [ ]:
# Obtener las columnas numéricas y categóricas después de la transformación
numeric_columns_transformed = pipeline.named_steps['preprocessor'].transformers_[0][1]['scaler'].get_feature_names_out(input_features=numeric_columns)
categorical_columns_transformed = pipeline.named_steps['preprocessor'].transformers_[1][1]['onehot'].get_feature_names_out(input_features=categorical_columns)

# Unir los nombres de las columnas numéricas y categóricas después de la transformación
all_columns_transformed = np.concatenate([numeric_columns_transformed, categorical_columns_transformed])

# Convertir el array transformado a un DataFrame con los nuevos nombres de columnas
transformed_df_2 = pd.DataFrame(transformed_df.toarray(), columns=all_columns_transformed)
transformed_df_2 


In [ ]:
transformed_df_2.to_csv('datasets/raw/transformed_df_2.csv')

In [ ]:
df_houses = transformed_df_2[['GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'OverallCond','Pesos', 'Alley_Grvl','Alley_Pave','Alley_nan', 'WallMat_Concrete','WallMat_Wood','WallMat_nan','SalePrice']]
plt.figure(figsize=(12, 10))
corr = df_houses.corr()
sns.heatmap(corr,
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        annot=True)

In [ ]:
corr

In [ ]:
# Definir las features (X) y la variable objetivo (y)
X = transformed_df_2[['GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'OverallCond', 'Pesos', 'Alley_Grvl', 'Alley_Pave', 'Alley_nan', 'WallMat_Concrete', 'WallMat_Wood', 'WallMat_nan']]
y = transformed_df_2['SalePrice']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el modelo de árboles de decisión
model = DecisionTreeRegressor(random_state=42)

# Evaluar el modelo mediante validación cruzada
cv_scores_mse = cross_val_score(model, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_scores_r2 = cross_val_score(model, X_train, y_train, cv=10, scoring='r2')

# Imprimir el rango del error cuadrático medio y del coeficiente de determinación
print(f"Error cuadrático medio: {-cv_scores_mse.max()} - {-cv_scores_mse.min()}")
print(f"Coeficiente de determinación (R^2): {cv_scores_r2.min()} - {cv_scores_r2.max()}")